# Interactive Backtest Visualization

This notebook demonstrates how to create interactive charts from backtest results using Plotly.

## Requirements

```bash
pip install plotly pandas pyarrow
```

In [ ]:
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go

# Set the results directory (modify this to your specific run)
# Example: run = Path('results/backtest_20250727_123456')
results_dir = Path("results")
run_dirs = [d for d in results_dir.iterdir() if d.is_dir()]
if run_dirs:
    run = max(run_dirs, key=lambda x: x.stat().st_mtime)  # Latest run
    print(f"Using results from: {run}")
else:
    print("No results directories found. Run a backtest first.")
    run = None

In [ ]:
if run:
    # Load data files
    data_file = run / "data.csv"
    trades_file = run / "trades.csv"
    events_file = run / "events.parquet"

    # Check which files exist
    print(f"Data file exists: {data_file.exists()}")
    print(f"Trades file exists: {trades_file.exists()}")
    print(f"Events file exists: {events_file.exists()}")

    if not data_file.exists():
        print(
            "\n⚠️ data.csv not found. Ensure backtest was run with visualization export enabled."
        )
        print("Add to your config: runtime.export_data_for_viz: true")

In [ ]:
if run and data_file.exists():
    # Load market data
    bars = pd.read_csv(data_file)
    print(f"Loaded {len(bars)} candles")
    print(f"Date range: {bars['timestamp'].min()} to {bars['timestamp'].max()}")

    # Show sample data
    print("\nSample data:")
    bars.head()

In [ ]:
if run and data_file.exists():
    # Create the candlestick chart
    fig = go.Figure(
        data=go.Candlestick(
            x=bars["timestamp"],
            open=bars["open"],
            high=bars["high"],
            low=bars["low"],
            close=bars["close"],
            name="Price",
        )
    )

    # Load and add FVG zones if available
    if events_file.exists():
        try:
            events = pd.read_parquet(events_file)
            fvg_events = (
                events[events["type"] == "FVGEvent"]
                if "type" in events.columns
                else pd.DataFrame()
            )

            print(f"Found {len(fvg_events)} FVG events")

            for i, event in fvg_events.iterrows():
                fig.add_shape(
                    type="rect",
                    x0=event["ts"],
                    x1=pd.to_datetime(event["ts"]) + pd.Timedelta("2H"),
                    y0=event["bottom"],
                    y1=event["top"],
                    fillcolor="rgba(100, 149, 237, 0.15)",
                    line={"width": 0},
                    name=f"FVG_{i}",
                )
        except Exception as e:
            print(f"Could not load events: {e}")

    # Load and add trades if available
    if trades_file.exists():
        try:
            trades = pd.read_csv(trades_file)
            print(f"Found {len(trades)} trades")

            # Entry points
            if "entry_ts" in trades.columns and "entry_price" in trades.columns:
                fig.add_trace(
                    go.Scatter(
                        x=trades["entry_ts"],
                        y=trades["entry_price"],
                        mode="markers",
                        name="Entry",
                        marker={"symbol": "triangle-up", "size": 12, "color": "lime"},
                    )
                )

            # Exit points
            if "exit_ts" in trades.columns and "exit_price" in trades.columns:
                fig.add_trace(
                    go.Scatter(
                        x=trades["exit_ts"],
                        y=trades["exit_price"],
                        mode="markers",
                        name="Exit",
                        marker={"symbol": "x", "size": 12, "color": "red"},
                    )
                )
        except Exception as e:
            print(f"Could not load trades: {e}")

    # Update layout
    fig.update_layout(
        height=800,
        title=f"Interactive Back-test Chart - {run.name}",
        xaxis_title="Time",
        yaxis_title="Price",
        showlegend=True,
        xaxis_rangeslider_visible=False,
    )

    # Display the chart
    fig.show()

## Using the Plot Builder Module

Alternatively, you can use the built-in plot builder for simplified chart creation:

In [1]:
# Using the plot builder module
try:
    from quant_algo.visual.plot_builder import build_plotly

    if run:
        # Create run context
        class RunContext:
            def __init__(self, out_dir):
                self.out_dir = Path(out_dir)
                self.data_path = self.out_dir / "data.csv"
                self.trades_path = self.out_dir / "trades.csv"
                self.events_path = self.out_dir / "events.parquet"

        run_ctx = RunContext(run)

        if run_ctx.data_path.exists():
            fig = build_plotly(run_ctx)
            fig.show()
        else:
            print("Data files not found for plot builder")

except ImportError as e:
    print(f"Plot builder not available: {e}")
    print("Make sure you're running from the project root directory")

Plot builder not available: No module named 'quant_algo'
Make sure you're running from the project root directory


## Customization Options

You can customize the visualization by:

1. **Filtering time ranges**: Use `bars = bars[bars['timestamp'] > '2025-01-01']`
2. **Changing FVG duration**: Modify the `pd.Timedelta('2H')` to your preferred duration
3. **Adding custom indicators**: Load your own data and add as traces
4. **Styling**: Modify colors, markers, and layout options

## Export Options

Save your charts:

In [ ]:
# Export options (uncomment to use)
if "fig" in locals():
    # Save as HTML
    # fig.write_html("backtest_chart.html")

    # Save as PNG (requires kaleido: pip install kaleido)
    # fig.write_image("backtest_chart.png", width=1600, height=900)

    # Save as PDF
    # fig.write_image("backtest_chart.pdf")

    print("Chart export options available (uncomment above lines to use)")